In [1]:
import sys
sys.path.insert(1, '../..')
import os
import re
import pandas as pd
from collections import Counter
from medicine import config
# from medicine.script.data_process import DataProcess

In [2]:
file_path = os.path.join(config.Path['data_dir'], '本草纲目(簡體)_Ver2.csv')
data = pd.read_csv(file_path, index_col=[0], engine='python', encoding='utf-8-sig')
data.columns

Index(['目錄', '篇名', '釋名', '內容', '部位', '部位內容', '主治', '氣味', '附方'], dtype='object')

In [3]:
compounds = data.loc[:,'附方']
compounds[11:13]

11    旧一，新一。辟禳时疫∶半天河水，饮之。（《医林集要》）身体白驳∶取树木 \n孔中水洗之，捣桂...
12                                                  NaN
Name: 附方, dtype: object

##### 觀察附方起始文字，並予以清理 (略)
1. 旧一，新一。
2. 新一。
3. 旧一。
4. 新入。
5. （原缺）

In [ ]:
# num_pattern = r'[一二三四五六七八九十]'
# old_pattern = r'旧' + num_pattern + r'{1,3}[\s。]*'
# new_pattern = r'新' + num_pattern + r'{1,3}[\s。]*'
# pattern = old_pattern + '.{1,2}' + new_pattern + \
#     '|' + old_pattern + \
#     '|' + new_pattern + \
#     '|' + '（原缺）' + \
#     '|' + '新入[\s。]*'
# # print(pattern)

# for i in range(len(compounds)):
#     compound = compounds[i]
#     if pd.isnull(compound):
#         continue
#     compound = compound.strip()
# #     # for test
# #     match = re.match(pattern, compound)
# #     if match:
# #         print(match.group())
# #         print(re.sub(pattern, '', compound) + '\n')
# #         pass
# #     else:
# #         print(re.sub(pattern, '', compound) + '\n')
#     compounds[i] = re.sub(pattern, '', compound)


In [4]:
d_len = 2
u_len = 35
# pattern = r'[。）》]{1,3}( ?\w{0,3} ?\n? ?\w{0,8}?[ ，、]{0,2}\w{1,11})∶'
# pattern = r'[。）》]{1,3}((?: ?\w{1,20}[，、]?\n?)?\
# (?:(?:\w+[，、]?){0,2}\n?){0,3}\w{1,11})∶'
# pattern = r'[。）》]{1,3}((?: ?\w{1,20}[ ，、]?\n?)?\
# (?:(?:\w+[，、]){0,2} ?\n?){0,3}\w{1,11})∶'
pattern = r'[。）》]((?: ?\w{0,10}[ ，、]?\n?){0,1}\
(?: ?\w{0,2}[，、] \n){0,2}\
(?:(?:\w+[，、]){1,3} ?\n?){0,3}\w{1,11})∶'


find_list = []
for compound in compounds:
    if pd.isnull(compound):
        continue
    finds = re.findall(pattern, compound)
    if len(finds) > 0:
        finds = [re.sub(r'\s', '', find) for find in finds]
        find_list.extend(finds)
find_list = [find for find in find_list if  d_len <= len(find) <= u_len]
print(len(set(find_list)))
# print(Counter(find_list))
# set(find_list)

5584


In [31]:
finds = []
for find in find_list:
    if re.search('^一.{0,5}$', find):
        finds.append(find)
set(finds)

{'一云',
 '一切下血',
 '一切丹毒',
 '一切冷漏',
 '一切反胃',
 '一切口疮',
 '一切壅滞',
 '一切头疮',
 '一切心痛',
 '一切恶疮',
 '一切折伤',
 '一切毒肿',
 '一切气痛',
 '一切泄痢',
 '一切泻痢',
 '一切漏疮',
 '一切漏疮有孔',
 '一切漏疾',
 '一切疔疮',
 '一切疔肿',
 '一切疮',
 '一切疮疖',
 '一切疮肿',
 '一切疳毒',
 '一切疳疮',
 '一切疳疾',
 '一切痈疮',
 '一切痈疽',
 '一切痈肿',
 '一切痈肿未溃',
 '一切痢下',
 '一切癣',
 '一切盗汗',
 '一切目疾',
 '一切眼疾',
 '一切眼翳',
 '一切积病',
 '一切积痢',
 '一切肿毒',
 '一切肿毒初起',
 '一切肿疾',
 '一切虚风',
 '一切虫病',
 '一切诸毒',
 '一切诸风',
 '一切顽风',
 '一切风疾',
 '一切风痰',
 '一切风证',
 '一切香港脚',
 '一剪金',
 '一方',
 '一法',
 '一粒金丹'}

In [5]:
print(pattern)

[。）》]((?: ?\w{0,10}[ ，、]?\n?){0,1}(?: ?\w{0,2}[，、] \n){0,2}(?:(?:\w+[，、]){1,3} ?\n?){0,3}\w{1,11})∶


In [6]:
compound_join = '|'.join(find_list)
pattern = '(' + compound_join + ')∶' + '(.*?)' + '(?=(?=(?:' + compound_join + ')∶)|(?<=.$))'

count = 0
for compound in compounds:
    if pd.isnull(compound):
        continue
    finds = re.findall(pattern, re.sub('\s', '', compound))
    if len(finds) > 0:
        print(compound.strip())
        print(finds)
        print()
        
    count += 1
    if count > 2:
        break


新一。灭瘢痕∶以冻凌频熨之，良。（《千金方》）
[('灭瘢痕', '以冻凌频熨之，良。（《千金方》）')]

旧一，新一。辟禳时疫∶半天河水，饮之。（《医林集要》）身体白驳∶取树木 
孔中水洗之，捣桂末唾和敷之，日再上。（张文仲《备急方》）
[('辟禳时疫', '半天河水，饮之。（《医林集要》）'), ('身体白驳', '取树木孔中水洗之，捣桂末唾和敷之，日再上。（张文仲《备急方》）')]



In [7]:
compound_df = pd.DataFrame(columns=['title', 'compound', 'content'])
compound_join = '|'.join(find_list)
pattern = '(' + compound_join + ')∶' + '(.*?)' + '(?=(?=(?:' + compound_join + ')∶)|(?<=.$))'

previous_title = None
for index, row in data.iterrows():
    title = row['篇名']
    compound = row['附方']
    if title == previous_title or pd.isnull(compound):
        continue
    else:
        previous_title = title
    finds = re.findall(pattern, re.sub('\s', '', compound))
    if len(finds) == 0:
        continue
    for find in finds:
        df_row = dict((k, '') for k in compound_df.columns)
        df_row['title'] = title
        df_row['compound'] = find[0]
        df_row['content'] = find[1]
        compound_df = compound_df.append(df_row, ignore_index=True)
    


In [8]:
# file_path = os.path.join(config.Path['data_dir'], '本草纲目(簡體)_Ver3.csv')
# compound_df.to_csv(file_path, encoding='utf-8-sig')

compound_df.head(3)

,title,compound,content
0,夏冰\n,灭瘢痕,以冻凌频熨之，良。（《千金方》）
1,半天河\n,辟禳时疫,半天河水，饮之。（《医林集要》）
2,半天河\n,身体白驳,取树木孔中水洗之，捣桂末唾和敷之，日再上。（张文仲《备急方》）


#### 去找尋附方內文中符合藥草名稱的pattern
#### <font color='red'>效果不佳</font>

In [14]:
from medicine.models.medicine import Medicine
from medicine.models.alias import Alias 

medicine_all = Medicine.query.all()
alias_all = Alias.query.all()

name_list = list()
for name_len in list(range(9, 1 , -1)):
    m_list = set([m_name.name for m_name in medicine_all if len(m_name.name) == name_len])
    name_list.extend(m_list)
    a_list = set([a_name.name for a_name in alias_all if len(a_name.name) == name_len])
    name_list.extend(a_list)

name_join = '|'.join(name_list)
pattern = '(?=(' + name_join + '))'
for index, row in compound_df.iterrows():
    content = re.sub('\s', '', row['content'])
    match = re.search('（\w{0,3}《?\w{2,8}》?）|《\w{2,8}》', content)
    if match:
        content = re.sub('（\w{0,3}《?\w{2,8}》?）|《\w{2,8}》', '', row['content'])
    find = re.findall(pattern, re.sub('\s', '', content))
#     print(row['title'], row['compound'], '\n', row['content'], '\n', set(find), '\n',)
#     print(row['compound'], '\n')



F:\Python\anaconda3\lib\site-packages\sqlalchemy\engine\default.py:470: Warning: (1366, "Incorrect string value: '\\xA5x\\xA5_\\xBC\\xD0...' for column 'VARIABLE_VALUE' at row 480")
  cursor.execute(statement, parameters)


In [ ]:
medicine_all = Medicine.query.all()
# m_set = set([medicine for medicine in medicine_all if len(medicine.name) == 1])
# sorted(m_set, key=lambda Medicine: Medicine.radical)

# [medicine for medicine in medicine_all if medicine.radical in ['草'] and len(medicine.name) == 1]
sorted([medicine for medicine in medicine_all if len(medicine.name) == 1], key=lambda Medicine: Medicine.radical)